In [1]:
!pip install scikit-image==0.14.2 && pip install numpy==1.15

    100% |████████████████████████████████| 25.3MB 1.9MB/s eta 0:00:01
    100% |████████████████████████████████| 808kB 32.7MB/s ta 0:00:01
  Found existing installation: dask 0.17.5
    Uninstalling dask-0.17.5:
      Successfully uninstalled dask-0.17.5
  Found existing installation: scikit-image 0.13.1
    Uninstalling scikit-image-0.13.1:
      Successfully uninstalled scikit-image-0.13.1
You are using pip version 10.0.1, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 13.9MB 3.5MB/s eta 0:00:01
aws-mxnet-mkl 1.6.0 has requirement numpy<2.0.0,>1.16.0, but you'll have numpy 1.15.0 which is incompatible.
  Found existing installation: numpy 1.16.4
    Uninstalling numpy-1.16.4:
      Successfully uninstalled numpy-1.16.4
You are using pip version 10.0.1, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [32]:
import boto3
import numpy as np
import time
import sagemaker

from skimage import io
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.mxnet import MXNet
from sagemaker.tuner import ContinuousParameter, HyperparameterTuner
from sagemaker.mxnet import MXNetModel

# Creating a model using MXNet

## Configure hyperparameters

In [33]:
# Number of output classes
num_classes = 2

# Batch size for training
mini_batch_size =  128

# Max epochs for training
epochs = 2

# Learning rate
learning_rate = 0.01

## Create a unique job name 

In [34]:
job_name_prefix = 'breast-cancer-detection'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
job_name = job_name_prefix + timestamp

## Specify the input paths for the job

In [35]:
bucket = 'nitinproject1'
input_prefix = 'breast-cancer-detection/input/recordio'
input_train = 's3://{}/{}/train/'.format(bucket, input_prefix)
input_test = 's3://{}/{}/test/'.format(bucket, input_prefix)

## Specify the output path for the job

In [36]:
output_prefix = 'breast-cancer-detection/output'
output_path = 's3://{}/{}/'.format(bucket, output_prefix)

## Configure training instances

In [37]:
instance_count = 1
instance_type = 'ml.p2.xlarge'
volume_size_gb = 50

## Get the execution role

In [38]:
role = get_execution_role()

## Configure train timeout

In [39]:
train_timeout = 360000

## Specify the path to the training script

In [40]:
training_script_path = 'mxnetScript.py'

## Create a sagemaker.MXNet estimator

In [41]:
estimator = MXNet(entry_point=training_script_path,
                  role=role,
                  train_instance_count=instance_count,
                  train_instance_type=instance_type,
                  train_volume_size=volume_size_gb,
                  train_max_run=train_timeout,
                  output_path=output_path,
                  framework_version='1.3.0',
                  py_version = 'py3',
                  hyperparameters = {
                    'num-classes': num_classes,
                    'mini-batch-size': mini_batch_size,
                    'epochs': epochs,
                    'learning-rate': learning_rate
                  },
                  metric_definitions = [
                     {
                         'Name': 'Validation-accuracy',
                         'Regex': 'validation: accuracy=([0-9\\.]+)'
                     }
                  ])

# Create a training job

In [13]:
estimator.fit({
    'train': input_train,
    'test': input_test
}, job_name = job_name)

2020-05-07 19:39:49 Starting - Starting the training job...
2020-05-07 19:39:51 Starting - Launching requested ML instances......
2020-05-07 19:41:04 Starting - Preparing the instances for training......
2020-05-07 19:42:19 Downloading - Downloading input data.........
2020-05-07 19:43:45 Training - Training image download completed. Training in progress..2020-05-07 19:43:46,210 sagemaker-containers INFO     Imported framework sagemaker_mxnet_container.training
2020-05-07 19:43:46,237 sagemaker_mxnet_container.training INFO     MXNet training environment: {'SM_INPUT_DATA_CONFIG': '{"test":{"RecordWrapperType":"None","S3DistributionType":"FullyReplicated","TrainingInputMode":"File"},"train":{"RecordWrapperType":"None","S3DistributionType":"FullyReplicated","TrainingInputMode":"File"}}', 'SM_OUTPUT_DATA_DIR': '/opt/ml/output/data', 'SM_NETWORK_INTERFACE_NAME': 'eth0', 'SM_USER_ARGS': '["--epochs","2","--learning-rate","0.01","--mini-batch-size","128","--num-classes","2"]', 'SM_RESOURCE_C

[19:44:09] src/io/iter_image_recordio_2.cc:170: ImageRecordIOParser2: /opt/ml/input/data/train/images_train.rec, use 1 threads for decoding..
[19:44:12] src/io/iter_image_recordio_2.cc:170: ImageRecordIOParser2: /opt/ml/input/data/test/images_test.rec, use 1 threads for decoding..
[19:44:16] src/operator/nn/./cudnn/./cudnn_algoreg-inl.h:109: Running performance tests to find the best convolution algorithm, this can take a while... (setting env variable MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable)
INFO:root:Epoch [0] Batch [1]#011Speed: 120.044629 samples/sec#011accuracy=0.742188
INFO:root:Epoch [0] Batch [2]#011Speed: 122.853896 samples/sec#011accuracy=0.718750
INFO:root:Epoch [0] Batch [3]#011Speed: 124.808827 samples/sec#011accuracy=0.716797
INFO:root:Epoch [0] Batch [4]#011Speed: 124.948196 samples/sec#011accuracy=0.715625
INFO:root:Epoch [0] Batch [5]#011Speed: 125.092074 samples/sec#011accuracy=0.726562
INFO:root:Epoch [0] Batch [6]#011Speed: 125.030342 samples/sec#011accuracy=0.

INFO:root:Epoch [0] Batch [84]#011Speed: 121.242698 samples/sec#011accuracy=0.824540
INFO:root:Epoch [0] Batch [85]#011Speed: 122.066635 samples/sec#011accuracy=0.824673
INFO:root:Epoch [0] Batch [86]#011Speed: 122.338701 samples/sec#011accuracy=0.824353
INFO:root:Epoch [0] Batch [87]#011Speed: 121.775665 samples/sec#011accuracy=0.824751
INFO:root:Epoch [0] Batch [88]#011Speed: 121.740761 samples/sec#011accuracy=0.825755
INFO:root:Epoch [0] Batch [89]#011Speed: 121.687229 samples/sec#011accuracy=0.826042
INFO:root:Epoch [0] Batch [90]#011Speed: 122.027321 samples/sec#011accuracy=0.826236
INFO:root:Epoch [0] Batch [91]#011Speed: 121.621179 samples/sec#011accuracy=0.826342
INFO:root:Epoch [0] Batch [92]#011Speed: 121.776245 samples/sec#011accuracy=0.826025
INFO:root:Epoch [0] Batch [93]#011Speed: 121.586832 samples/sec#011accuracy=0.825964
INFO:root:Epoch [0] Batch [94]#011Speed: 122.073213 samples/sec#011accuracy=0.825987
INFO:root:Epoch [0] Batch [95]#011Speed: 121.869734 samples/sec#0

INFO:root:Epoch [0] Batch [180]#011Speed: 121.085246 samples/sec#011accuracy=0.839995
INFO:root:Epoch [0] Batch [181]#011Speed: 121.633827 samples/sec#011accuracy=0.839672
INFO:root:Epoch [0] Batch [182]#011Speed: 121.354815 samples/sec#011accuracy=0.839822
INFO:root:Epoch [0] Batch [183]#011Speed: 121.771439 samples/sec#011accuracy=0.839844
INFO:root:Epoch [0] Batch [184]#011Speed: 121.113272 samples/sec#011accuracy=0.839949
INFO:root:Epoch [0] Batch [185]#011Speed: 121.759453 samples/sec#011accuracy=0.840180
INFO:root:Epoch [0] Batch [186]#011Speed: 121.331366 samples/sec#011accuracy=0.840282
INFO:root:Epoch [0] Batch [187]#011Speed: 121.614071 samples/sec#011accuracy=0.840218
INFO:root:Epoch [0] Batch [188]#011Speed: 121.295264 samples/sec#011accuracy=0.839947
INFO:root:Epoch [0] Batch [189]#011Speed: 121.839836 samples/sec#011accuracy=0.840255
INFO:root:Epoch [0] Batch [190]#011Speed: 121.462001 samples/sec#011accuracy=0.840519
INFO:root:Epoch [0] Batch [191]#011Speed: 121.160804 s

INFO:root:Epoch [0] Batch [275]#011Speed: 120.877260 samples/sec#011accuracy=0.842929
INFO:root:Epoch [0] Batch [276]#011Speed: 121.699118 samples/sec#011accuracy=0.843017
INFO:root:Epoch [0] Batch [277]#011Speed: 120.819754 samples/sec#011accuracy=0.843104
INFO:root:Epoch [0] Batch [278]#011Speed: 121.380359 samples/sec#011accuracy=0.843190
INFO:root:Epoch [0] Batch [279]#011Speed: 120.894681 samples/sec#011accuracy=0.843025
INFO:root:Epoch [0] Batch [280]#011Speed: 121.110431 samples/sec#011accuracy=0.842972
INFO:root:Epoch [0] Batch [281]#011Speed: 121.126333 samples/sec#011accuracy=0.842753
INFO:root:Epoch [0] Batch [282]#011Speed: 121.144345 samples/sec#011accuracy=0.842646
INFO:root:Epoch [0] Batch [283]#011Speed: 120.993229 samples/sec#011accuracy=0.842760
INFO:root:Epoch [0] Batch [284]#011Speed: 121.102727 samples/sec#011accuracy=0.842736
INFO:root:Epoch [0] Batch [285]#011Speed: 120.932288 samples/sec#011accuracy=0.842767
INFO:root:Epoch [0] Batch [286]#011Speed: 121.094723 s

...

INFO:root:Epoch [1] Batch [1101]#011Speed: 120.357994 samples/sec#011accuracy=0.875624
INFO:root:Epoch [1] Batch [1102]#011Speed: 120.535556 samples/sec#011accuracy=0.875602
INFO:root:Epoch [1] Batch [1103]#011Speed: 120.337949 samples/sec#011accuracy=0.875637
INFO:root:Epoch [1] Batch [1104]#011Speed: 119.871747 samples/sec#011accuracy=0.875622
INFO:root:Epoch [1] Batch [1105]#011Speed: 120.334254 samples/sec#011accuracy=0.875657
INFO:root:Epoch [1] Batch [1106]#011Speed: 119.982307 samples/sec#011accuracy=0.875678
INFO:root:Epoch [1] Batch [1107]#011Speed: 120.801539 samples/sec#011accuracy=0.875691
INFO:root:Epoch [1] Batch [1108]#011Speed: 120.532742 samples/sec#011accuracy=0.875655
INFO:root:Epoch [1] Batch [1109]#011Speed: 120.648158 samples/sec#011accuracy=0.875619
INFO:root:Epoch [1] Batch [1110]#011Speed: 120.052521 samples/sec#011accuracy=0.875633
INFO:root:Epoch [1] Batch [1111]#011Speed: 120.104731 samples/sec#011accuracy=0.875646
INFO:root:Epoch [1] Batch [1112]#011Speed: 

INFO:root:Epoch [1] Batch [1187]#011Speed: 120.827666 samples/sec#011accuracy=0.875612
INFO:root:Epoch [1] Batch [1188]#011Speed: 120.900289 samples/sec#011accuracy=0.875604
INFO:root:Epoch [1] Batch [1189]#011Speed: 120.172990 samples/sec#011accuracy=0.875611
INFO:root:Epoch [1] Batch [1190]#011Speed: 120.787896 samples/sec#011accuracy=0.875623
INFO:root:Epoch [1] Batch [1191]#011Speed: 120.347741 samples/sec#011accuracy=0.875596
INFO:root:Epoch [1] Batch [1192]#011Speed: 120.412442 samples/sec#011accuracy=0.875563
INFO:root:Epoch [1] Batch [1193]#011Speed: 120.638941 samples/sec#011accuracy=0.875517
INFO:root:Epoch [1] Batch [1194]#011Speed: 120.385117 samples/sec#011accuracy=0.875490
INFO:root:Epoch [1] Batch [1195]#011Speed: 120.269018 samples/sec#011accuracy=0.875503
INFO:root:Epoch [1] Batch [1196]#011Speed: 120.511855 samples/sec#011accuracy=0.875509
INFO:root:Epoch [1] Batch [1197]#011Speed: 120.479645 samples/sec#011accuracy=0.875496
INFO:root:Epoch [1] Batch [1198]#011Speed: 

INFO:root:Epoch [1] Batch [1282]#011Speed: 120.554448 samples/sec#011accuracy=0.876090
INFO:root:Epoch [1] Batch [1283]#011Speed: 120.242943 samples/sec#011accuracy=0.876083
INFO:root:Epoch [1] Batch [1284]#011Speed: 120.830359 samples/sec#011accuracy=0.876100
INFO:root:Epoch [1] Batch [1285]#011Speed: 120.538587 samples/sec#011accuracy=0.876112
INFO:root:Epoch [1] Batch [1286]#011Speed: 120.504795 samples/sec#011accuracy=0.876062
INFO:root:Epoch [1] Batch [1287]#011Speed: 120.627366 samples/sec#011accuracy=0.876074
INFO:root:Epoch [1] Batch [1288]#011Speed: 120.639808 samples/sec#011accuracy=0.876097
INFO:root:Epoch [1] Batch [1289]#011Speed: 120.371594 samples/sec#011accuracy=0.876078
INFO:root:Epoch [1] Batch [1290]#011Speed: 119.722479 samples/sec#011accuracy=0.876071
INFO:root:Epoch [1] Batch [1291]#011Speed: 120.580117 samples/sec#011accuracy=0.876034
INFO:root:Epoch [1] Batch [1292]#011Speed: 119.984747 samples/sec#011accuracy=0.876045
INFO:root:Epoch [1] Batch [1293]#011Speed: 

INFO:root:Epoch [1] Batch [1376]#011Speed: 120.239173 samples/sec#011accuracy=0.876515
INFO:root:Epoch [1] Batch [1377]#011Speed: 120.536936 samples/sec#011accuracy=0.876531
INFO:root:Epoch [1] Batch [1378]#011Speed: 120.474806 samples/sec#011accuracy=0.876513
INFO:root:Epoch [1] Batch [1379]#011Speed: 120.531172 samples/sec#011accuracy=0.876489
INFO:root:Epoch [1] Batch [1380]#011Speed: 120.594770 samples/sec#011accuracy=0.876510
INFO:root:Epoch [1] Batch [1381]#011Speed: 120.338731 samples/sec#011accuracy=0.876555
INFO:root:Epoch [1] Batch [1382]#011Speed: 120.440049 samples/sec#011accuracy=0.876576
INFO:root:Epoch [1] Batch [1383]#011Speed: 119.863210 samples/sec#011accuracy=0.876586
INFO:root:Epoch [1] Batch [1384]#011Speed: 120.670259 samples/sec#011accuracy=0.876625
INFO:root:Epoch [1] Batch [1385]#011Speed: 120.440103 samples/sec#011accuracy=0.876652
INFO:root:Epoch [1] Batch [1386]#011Speed: 120.260639 samples/sec#011accuracy=0.876673
INFO:root:Epoch [1] Batch [1387]#011Speed: 

INFO:root:Epoch [1] Batch [1463]#011Speed: 120.279580 samples/sec#011accuracy=0.876942
INFO:root:Epoch [1] Batch [1464]#011Speed: 119.639025 samples/sec#011accuracy=0.876973
INFO:root:Epoch [1] Batch [1465]#011Speed: 120.242000 samples/sec#011accuracy=0.876972
INFO:root:Epoch [1] Batch [1466]#011Speed: 119.289537 samples/sec#011accuracy=0.876954
INFO:root:Epoch [1] Batch [1467]#011Speed: 120.435159 samples/sec#011accuracy=0.876964
INFO:root:Epoch [1] Batch [1468]#011Speed: 120.016612 samples/sec#011accuracy=0.876968
INFO:root:Epoch [1] Batch [1469]#011Speed: 119.703793 samples/sec#011accuracy=0.876998
INFO:root:Epoch [1] Batch [1470]#011Speed: 120.301007 samples/sec#011accuracy=0.877002
INFO:root:Epoch [1] Batch [1471]#011Speed: 119.788967 samples/sec#011accuracy=0.876969
INFO:root:Epoch [1] Batch [1472]#011Speed: 120.380312 samples/sec#011accuracy=0.876946
INFO:root:Epoch [1] Batch [1473]#011Speed: 120.043260 samples/sec#011accuracy=0.876940
INFO:root:Epoch [1] Batch [1474]#011Speed: 

# Creating a tuning job

## Defining tuning configuration

In [14]:
hyperparameter_ranges = {
    'learning-rate': ContinuousParameter(0.001, 1.0)
}
objective_metric_name = 'Validation-accuracy'
objective_type = 'Maximize'

max_jobs=2
max_parallel_jobs=2

## Create a unique job name

In [15]:
job_name_prefix = 'bcd-tuning'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
job_name = job_name_prefix + timestamp

## Creating a hyperparameter tuner

In [16]:
tuner = HyperparameterTuner(estimator=estimator, 
                            objective_metric_name=objective_metric_name, 
                            hyperparameter_ranges=hyperparameter_ranges,
                            objective_type=objective_type, 
                            max_jobs=max_jobs, 
                            max_parallel_jobs=max_parallel_jobs,
                            metric_definitions = [
                                 {
                                     'Name': 'Validation-accuracy',
                                     'Regex': 'validation: accuracy=([0-9\\.]+)'
                                 }
                            ])

In [17]:
tuner.fit({
    'train': input_train,
    'test': input_test
}, job_name = job_name)
tuner.wait()

............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................!


# Deploying the best model found by the tuning job

## Get the execution role

In [42]:
role = get_execution_role()

## Configure hosting instances

In [43]:
instance_count = 1
instance_type = 'ml.m4.xlarge'

## Specify the path to the training script

In [44]:
training_script_path = 'mxnetScript.py'

## Create a unique model name

In [45]:
model_name_prefix = 'bcd-image-classification-mxnet'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
model_name = model_name_prefix + timestamp

## Create a MXNetModel object

In [46]:
model_artifacts_s3_path = 's3://nitinproject1/breast-cancer-detection/output/bcd-tuning-2020-05-07-20-52-13-002-dd592fbc/output/model.tar.gz'
model = MXNetModel(
    name=model_name,
    model_data=model_artifacts_s3_path,
    role=role,
    py_version='py3',
    entry_point=training_script_path
)

No framework_version specified, defaulting to version 1.2. This is not the latest supported version. If you would like to use version 1.6.0, please add framework_version=1.6.0 to your constructor.


## Create a unique endpoint name

In [47]:
endpoint_name_prefix = 'breast-cancer-detection-ep'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_name = endpoint_name_prefix + timestamp

## Create a model, an endpoint configuration and an endpoint

In [48]:
predictor = model.deploy(
    endpoint_name=endpoint_name,
    initial_instance_count=instance_count,
    instance_type=instance_type
)

---------------!

# Testing the deployed model

In [49]:
def read_image(filename):
    image = io.imread(filename)
    image = np.array(image).transpose(2, 0, 1)
    image = np.expand_dims(image, axis=0)
    return image

In [50]:
def predict_breast_cancer(filename):
    image = read_image(filename)
    predicted_class = predictor.predict(image)
    if predicted_class == 0:
        print('Breast cancer not detected')
    else:
        print('Breast cancer detected')

In [51]:
predict_breast_cancer('images/0/8975_idx5_x2851_y1201_class0.png')

Breast cancer not detected


In [56]:
predict_breast_cancer('images/1/10273_idx5_x1801_y851_class1.png')

Breast cancer detected


In [57]:
predict_breast_cancer('samples/fake_images-main.png')

predict_breast_cancer('samples/fake_images-1.png')

predict_breast_cancer('samples/fake_images-2.png')

predict_breast_cancer('samples/fake_images-3.png')

predict_breast_cancer('samples/fake_images-4.png')

predict_breast_cancer('samples/fake_images-5.png')

predict_breast_cancer('samples/fake_images-6.png')

predict_breast_cancer('samples/real_images.png')

Breast cancer detected
Breast cancer detected
Breast cancer not detected
Breast cancer not detected
Breast cancer not detected
Breast cancer not detected
Breast cancer not detected
Breast cancer detected


# Deleting endpoint

In [27]:
sagemaker.Session().delete_endpoint(predictor.endpoint)